In [ ]:
import sys
sys.path.insert(0, '../')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
import io
import tabulate
import helpers_eda_inicial
import helpers_cbc

Primero me traigo todos los datasets

In [ ]:
cbc, df_carreras = helpers_cbc.get_data('../../../assets/bronze/CBC/Sitacad_Tesis (1).xlsx')
actas = pd.read_csv('../../../assets/bronze/FCEN/FCEN_oficial_v2/reportes_actas.csv')
personas = pd.read_csv('../../../assets/bronze/FCEN/FCEN_oficial_v3/reporte_personas.csv')

Cambio el tipo de dato del DNI en el CBC para que coincida con el tipo de datos de la FCEN

In [3]:
cbc['Dni'] = cbc['Dni'].astype(str)
actas['fecha'] = pd.to_datetime(actas['fecha'], format='%Y-%m-%d')
actas['año'] = actas['fecha'].dt.year
actas['mes'] = actas['fecha'].dt.month

En función de lo trabajado en `packages/exploratory-data-analysis/FCEN/02-eda_buscando_target.ipynb` decidimos que vamos a trabajar con los inscriptos en 2022. Por lo tanto, me quedo con esos datos.

In [ ]:
personas.columns

Vemos la cantidad de inscriptos por año

In [ ]:
personas['año_inscripcion_facultad'].value_counts().sort_index()

In [ ]:
print(personas.groupby('carrera_principal')['año_inscripcion_facultad'].value_counts().sort_index().to_markdown())

Tomamos solamente las personas de 2022

In [7]:
personas_2022 = personas[personas['año_inscripcion_facultad'] == 2022]

Me guardo la lista de DNIs de las personas de 2022

In [8]:
personas_2022_dnis = personas_2022['dni'].unique()

In [ ]:
personas_2022_dnis.size

Nos guardamos las actas de 2022

In [10]:
actas_inscriptos_2022 = actas[actas['dni'].isin(personas_2022_dnis)]

In [ ]:
actas_inscriptos_2022['dni'].nunique()

Veo además que hay personas con actas anteriores a 2022, lo cual es extraño, pero podrían ser datos mal ingresados. El problema es que no son todas personas distinnas con los datos mal ingresados.

In [ ]:
actas_inscriptos_2022['año'].value_counts().sort_index()

In [ ]:
actas_inscriptos_2022[actas_inscriptos_2022['año'] < 2022]['dni'].unique()

In [ ]:
actas[actas['dni'].isin(actas_inscriptos_2022[actas_inscriptos_2022['año'] < 2022]['dni'].unique())].groupby('dni')['fecha'].max()

Todos excepto por 1 tienen registros posteriores a 2022, entonces por ahora elimino los registros anteriores a ese año.

In [15]:
actas_inscriptos_2022 = actas_inscriptos_2022[actas_inscriptos_2022['año'] > 2021]

In [ ]:
actas_inscriptos_2022['dni'].nunique()

Defino los semestres relativos

In [17]:
conditions = [
    (actas_inscriptos_2022['año'] == 2022) & (actas_inscriptos_2022['mes'].between(1, 8)),
    (actas_inscriptos_2022['año'] == 2022) & (actas_inscriptos_2022['mes'].between(9, 12)),
    (actas_inscriptos_2022['año'] == 2023) & (actas_inscriptos_2022['mes'].isin([1, 2])),
    (actas_inscriptos_2022['año'] == 2023) & (actas_inscriptos_2022['mes'].between(3, 8)),
    (actas_inscriptos_2022['año'] == 2023) & (actas_inscriptos_2022['mes'].between(9, 12)),
    (actas_inscriptos_2022['año'] == 2024) & (actas_inscriptos_2022['mes'].isin([1, 2])),
    (actas_inscriptos_2022['año'] == 2024) & (actas_inscriptos_2022['mes'].between(3, 8)),
    (actas_inscriptos_2022['año'] == 2024) & (actas_inscriptos_2022['mes'].between(9, 12)),
    (actas_inscriptos_2022['año'] == 2025)
]
values = [0, 1, 1, 2, 3, 3, 4, 5, 5]

actas_inscriptos_2022['semestre_relativo'] = np.select(conditions, values, default=np.nan)

In [ ]:
actas_inscriptos_2022['semestre_relativo'].value_counts(dropna = False)

In [ ]:
actas_inscriptos_2022

Cuento la cantidad de personas que no tienen un aprobado o que tampoco se anotaron a nada.

In [20]:
quinto_semestre = actas_inscriptos_2022[actas_inscriptos_2022['semestre_relativo'] == 5]

In [21]:
actas_quinto_semestre = personas_2022.merge(quinto_semestre, on='dni', how='left')

In [ ]:
actas_quinto_semestre.columns

In [ ]:
actas_quinto_semestre['materia'].isna().sum()

Veamos la cantidad de gente en personas_2022 que no se encuentra en actas_inscriptos_2022

In [ ]:
personas_2022_dnis.size

In [ ]:
np.isin(personas_2022_dnis, actas_inscriptos_2022['dni'].unique()).sum()

In [ ]:
# 1. Obtener todas las combinaciones posibles de DNI y nuevo_valor
personas_dni = personas_2022['dni'].unique()
periodos = actas_inscriptos_2022['semestre_relativo'].dropna().unique()
combinaciones = pd.MultiIndex.from_product([personas_dni, periodos], names=['dni', 'semestre_relativo']).to_frame(index=False)

# 2. Crear una tabla que indique en qué combinaciones hubo al menos una materia aprobada
aprobaciones = actas_inscriptos_2022[actas_inscriptos_2022['resultado'] == 'Aprobado'][['dni', 'semestre_relativo']].drop_duplicates()
aprobaciones['aprobo'] = True

# 3. Merge: unimos todas las combinaciones con las aprobaciones
merged_2022 = combinaciones.merge(aprobaciones, on=['dni', 'semestre_relativo'], how='left')

# 4. Marcar las combinaciones sin materias aprobadas
merged_2022['no_aprobo'] = merged_2022['aprobo'].isna()

# 5. Contar por periodo
conteo = merged_2022.groupby('semestre_relativo')['no_aprobo'].sum().reset_index()

# 6. Graficar
plt.figure(figsize=(8,5))
plt.bar(conteo['semestre_relativo'], conteo['no_aprobo'], color='tomato')
plt.xlabel('Periodo')
plt.ylabel('Cantidad de personas sin materias aprobadas')
plt.title('Cantidad de personas sin aprobar (ni presentarse) por periodo')
plt.xticks(conteo['semestre_relativo'])
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

En realidad nos gustaría tener ese grafico dividido para la cohorte 2022C1 y 2022C2. La forma en que definimos dichas cohortes es:

- Tomamos como cohorte 2022 a los estudiantes con LU 2022 y que tengan algún registro entre enero a agosto 2022
- Tomamos como cohorte 2022 a los estudiantes con LU 2022 y que NO tengan algún registro entre enero a agosto 2022 pero sí alguno a partir de septiembre 2022

In [ ]:
cohorte_20221c = actas_inscriptos_2022[actas_inscriptos_2022['semestre_relativo'] == 0]['dni'].unique()
actas_inscriptos_2022_cohorte_20221c = actas_inscriptos_2022[actas_inscriptos_2022['dni'].isin(cohorte_20221c)].copy()
actas_inscriptos_2022_cohorte_20221c.rename(columns={'semestre_relativo': 'semestre_relativo_20221c'}, inplace=True)
cohorte_20221c.size

In [ ]:
cohorte_20222c = actas_inscriptos_2022[~actas_inscriptos_2022['dni'].isin(cohorte_20221c)]['dni'].unique()
actas_inscriptos_2022_cohorte_20222c = actas_inscriptos_2022[actas_inscriptos_2022['dni'].isin(cohorte_20222c)].copy()
cohorte_20222c.size

In [29]:
conditions = [
    (actas_inscriptos_2022_cohorte_20222c['año'] == 2022) & (actas_inscriptos_2022_cohorte_20222c['mes'].between(9, 12)),
    (actas_inscriptos_2022_cohorte_20222c['año'] == 2023) & (actas_inscriptos_2022_cohorte_20222c['mes'].isin([1, 2])),
    (actas_inscriptos_2022_cohorte_20222c['año'] == 2023) & (actas_inscriptos_2022_cohorte_20222c['mes'].between(3, 8)),
    (actas_inscriptos_2022_cohorte_20222c['año'] == 2023) & (actas_inscriptos_2022_cohorte_20222c['mes'].between(9, 12)),
    (actas_inscriptos_2022_cohorte_20222c['año'] == 2024) & (actas_inscriptos_2022_cohorte_20222c['mes'].isin([1, 2])),
    (actas_inscriptos_2022_cohorte_20222c['año'] == 2024) & (actas_inscriptos_2022_cohorte_20222c['mes'].between(3, 8)),
    (actas_inscriptos_2022_cohorte_20222c['año'] == 2024) & (actas_inscriptos_2022_cohorte_20222c['mes'].between(9, 12)),
    (actas_inscriptos_2022_cohorte_20222c['año'] == 2025)
]
values = [0, 0, 1, 2, 2, 3, 4, 4]

actas_inscriptos_2022_cohorte_20222c['semestre_relativo_20222c'] = np.select(conditions, values, default=np.nan)

In [ ]:
# 1. Filtrar los DNI que *sí* tienen al menos una fila con semestre_relativo_20222c == 0
dnis_con_semestre_0 = actas_inscriptos_2022_cohorte_20222c.loc[
    actas_inscriptos_2022_cohorte_20222c['semestre_relativo_20222c'] == 0, 'dni'
].unique()

# 2. Filtrar los DNI que *no* están en esa lista
dnis_sin_semestre_0 = actas_inscriptos_2022_cohorte_20222c.loc[
    ~actas_inscriptos_2022_cohorte_20222c['dni'].isin(dnis_con_semestre_0), 'dni'
].unique()

# 3. Mostrar los resultados
print(dnis_sin_semestre_0)

In [ ]:
actas_inscriptos_2022_cohorte_20222c[actas_inscriptos_2022_cohorte_20222c['dni'].isin(dnis_sin_semestre_0)]['semestre_relativo_20222c'].value_counts(dropna = False)

In [ ]:
# 1. Filtrar los DNI que *sí* tienen al menos una fila con semestre_relativo_20222c == 0
dnis_con_semestre_1 = actas_inscriptos_2022_cohorte_20222c.loc[
    actas_inscriptos_2022_cohorte_20222c['semestre_relativo_20222c'] == 1, 'dni'
].unique()

# 3. Mostrar los resultados
np.isin(dnis_sin_semestre_0, dnis_con_semestre_1)

In [ ]:
actas_inscriptos_2022_cohorte_20222c[(actas_inscriptos_2022_cohorte_20222c['dni'].isin(dnis_sin_semestre_0)) & (actas_inscriptos_2022_cohorte_20222c['semestre_relativo_20222c'] == 1)]['fecha'].value_counts(dropna = False)

Me quedo solamente con la gente que tiene inscripciones en el segundo semestre de 2022

In [34]:
actas_inscriptos_2022_cohorte_20222c = actas_inscriptos_2022_cohorte_20222c[~actas_inscriptos_2022_cohorte_20222c['dni'].isin(dnis_sin_semestre_0)]

In [ ]:
actas_inscriptos_2022_cohorte_20222c['dni'].nunique()

In [36]:
cohorte_20222c = actas_inscriptos_2022_cohorte_20222c['dni'].unique()

In [ ]:
actas_inscriptos_2022['dni'].nunique()

In [ ]:
personas_2022_dnis.size - actas_inscriptos_2022['dni'].nunique()

In [ ]:
personas_2022[~personas_2022['dni'].isin(actas_inscriptos_2022['dni'].unique())]

In [ ]:
# 1. Obtener todas las combinaciones posibles de DNI y nuevo_valor
periodos = actas_inscriptos_2022_cohorte_20221c['semestre_relativo_20221c'].dropna().unique()
combinaciones_cohorte_20221c = pd.MultiIndex.from_product([cohorte_20221c, periodos], names=['dni', 'semestre_relativo_20221c']).to_frame(index=False)

# 2. Crear una tabla que indique en qué combinaciones hubo al menos una materia aprobada
aprobaciones = actas_inscriptos_2022_cohorte_20221c[actas_inscriptos_2022_cohorte_20221c['resultado'] == 'Aprobado'][['dni', 'semestre_relativo_20221c']].drop_duplicates()
aprobaciones['aprobo'] = True

# 3. Merge: unimos todas las combinaciones con las aprobaciones
merged = combinaciones_cohorte_20221c.merge(aprobaciones, on=['dni', 'semestre_relativo_20221c'], how='left')

# 4. Marcar las combinaciones sin materias aprobadas
merged['no_aprobo'] = merged['aprobo'].isna()

# 5. Contar por periodo
conteo = merged.groupby('semestre_relativo_20221c')['no_aprobo'].sum().reset_index()

# 6. Graficar
plt.figure(figsize=(8,5))
plt.bar(conteo['semestre_relativo_20221c'], conteo['no_aprobo'], color='tomato')
plt.xlabel('Periodo')
plt.ylabel('Cantidad de personas sin materias aprobadas')
plt.title('Cantidad de personas sin aprobar (ni presentarse) por periodo cohorte 20221c')
plt.xticks(conteo['semestre_relativo_20221c'])
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
conteo

In [ ]:
# 6. Graficar
plt.figure(figsize=(8,5))
plt.bar(conteo[conteo['semestre_relativo_20221c'] < 5]['semestre_relativo_20221c'], conteo[conteo['semestre_relativo_20221c'] < 5]['no_aprobo'], color='tomato')
plt.xlabel('Periodo')
plt.ylabel('Cantidad de personas sin materias aprobadas')
plt.title('Cantidad de personas sin aprobar (ni presentarse) por periodo cohorte 20221c')
plt.xticks(conteo[conteo['semestre_relativo_20221c'] < 5]['semestre_relativo_20221c'])
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# 1. Obtener todas las combinaciones posibles de DNI y nuevo_valor
periodos = actas_inscriptos_2022_cohorte_20222c['semestre_relativo_20222c'].dropna().unique()
combinaciones_cohorte_20222c = pd.MultiIndex.from_product([cohorte_20222c, periodos], names=['dni', 'semestre_relativo_20222c']).to_frame(index=False)

# 2. Crear una tabla que indique en qué combinaciones hubo al menos una materia aprobada
aprobaciones = actas_inscriptos_2022_cohorte_20222c[actas_inscriptos_2022_cohorte_20222c['resultado'] == 'Aprobado'][['dni', 'semestre_relativo_20222c']].drop_duplicates()
aprobaciones['aprobo'] = True

# 3. Merge: unimos todas las combinaciones con las aprobaciones
merged = combinaciones_cohorte_20222c.merge(aprobaciones, on=['dni', 'semestre_relativo_20222c'], how='left')

# 4. Marcar las combinaciones sin materias aprobadas
merged['no_aprobo'] = merged['aprobo'].isna()

# 5. Contar por periodo
conteo = merged.groupby('semestre_relativo_20222c')['no_aprobo'].sum().reset_index()

# 6. Graficar
plt.figure(figsize=(8,5))
plt.bar(conteo['semestre_relativo_20222c'], conteo['no_aprobo'], color='tomato')
plt.xlabel('Periodo')
plt.ylabel('Cantidad de personas sin materias aprobadas')
plt.title('Cantidad de personas sin aprobar (ni presentarse) por periodo 20222c')
plt.xticks(conteo['semestre_relativo_20222c'])
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

Si miramos la cantidad de gente 'nueva' que no tiene un aprobado, la cantidad es bastante reducida.

In [ ]:
# Asegurarnos que nuevo_valor es numérico para ordenarlo
merged_2022['semestre_relativo'] = merged_2022['semestre_relativo'].astype(int)

# Crear un diccionario para guardar los nuevos por periodo
nuevos_no_aprobaron = {}

# Mantener un set de DNIs que ya habían fallado antes
historico_no_aprobados = set()

# Iterar por los periodos en orden
for periodo in sorted(merged_2022['semestre_relativo'].unique()):
    # Personas que no aprobaron en este periodo
    actuales = set(merged_2022[(merged_2022['semestre_relativo'] == periodo) & (merged_2022['no_aprobo'])]['dni'])
    
    # Nuevos = actuales que no estaban antes
    nuevos = actuales - historico_no_aprobados
    nuevos_no_aprobaron[periodo] = len(nuevos)
    
    # Actualizar el histórico
    historico_no_aprobados.update(actuales)

# Convertir a DataFrame para graficar o analizar
nuevos_df = pd.DataFrame(list(nuevos_no_aprobaron.items()), columns=['semestre_relativo', 'nuevos_no_aprobaron'])
print(nuevos_df)

## 3 variables nuevas

Por cada dni quiero calcular para cada valor de 'semestre_relativo':
- #inscripciones: cantidad de filas con tipo_acta 'Acta de Regulares/Promociones' (O sea, materia) que tiene con ese semestre, no importa si aprobó, reprobó u abandonó
- #tp_aprobados: debe tener en 'tipo_acta' el valor 'Acta de Regulares/Promociones' y en 'resultado' el valor de 'Aprobado'
- #finales_inscriptos: cantidad de filas con el valor 'Acta de Examen' en la columna 'tipo_acta'

In [45]:
# Crear columnas auxiliares
actas_inscriptos_2022['es_inscripcion'] = (
    (actas_inscriptos_2022['tipo_acta'] == 'Acta de Regulares/Promociones')
)

actas_inscriptos_2022['es_tp_aprobado'] = (
    (actas_inscriptos_2022['tipo_acta'] == 'Acta de Regulares/Promociones') &
    (actas_inscriptos_2022['resultado'] == 'Aprobado')
)

actas_inscriptos_2022['es_final_inscripto'] = (
    actas_inscriptos_2022['tipo_acta'] == 'Acta de Examen'
)

# Agrupar y calcular métricas
resumen_v2 = actas_inscriptos_2022.groupby(['dni', 'semestre_relativo']).agg(
    inscripciones=('es_inscripcion', 'sum'),
    tp_aprobados=('es_tp_aprobado', 'sum'),
    finales_inscriptos=('es_final_inscripto', 'sum')
).reset_index()

# 5. Hacer el merge con el resumen
resumen_completo = combinaciones.merge(
    resumen_v2,
    on=['dni', 'semestre_relativo'],
    how='left'
).fillna(0)

In [ ]:
resumen_completo.groupby('semestre_relativo').count()

In [ ]:
resumen_completo

In [ ]:
metricas = ['inscripciones', 'tp_aprobados', 'finales_inscriptos']

for metrica in metricas:
    tabla = pd.crosstab(resumen_completo['semestre_relativo'], resumen_completo[metrica])
    plt.figure(figsize=(10, 6))
    sns.heatmap(tabla, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Distribución de {metrica} por semestre_relativo')
    plt.xlabel(metrica)
    plt.ylabel('semestre_relativo')
    plt.show()

In [ ]:
resumen_completo['total_actividad'] = (
    resumen_completo['inscripciones'] +
    resumen_completo['tp_aprobados'] +
    resumen_completo['finales_inscriptos']
)

tabla = pd.crosstab(resumen_completo['semestre_relativo'], resumen_completo['total_actividad'])

plt.figure(figsize=(10, 6))
sns.heatmap(tabla, annot=True, fmt='d', cmap='Purples')
plt.title('Distribución de actividad total por semestre_relativo')
plt.xlabel('Total de Actividad (inscripciones + tp_aprobados + finales_inscriptos)')
plt.ylabel('semestre_relativo')
plt.show()

Ahora lo vemos por cohorte

In [50]:
def cuatro_metricas(actas, combinaciones, cohorte):
    # Crear columnas auxiliares
    actas['es_inscripcion'] = (
        (actas['tipo_acta'] == 'Acta de Regulares/Promociones')
    )

    actas['es_tp_aprobado'] = (
        (actas['tipo_acta'] == 'Acta de Regulares/Promociones') &
        (actas['resultado'] == 'Aprobado')
    )

    actas['es_final_inscripto'] = (
        actas['tipo_acta'] == 'Acta de Examen'
    )

    # Agrupar y calcular métricas
    resumen_v2 = actas.groupby(['dni', f'semestre_relativo_{cohorte}']).agg(
        inscripciones=('es_inscripcion', 'sum'),
        tp_aprobados=('es_tp_aprobado', 'sum'),
        finales_inscriptos=('es_final_inscripto', 'sum')
    ).reset_index()

    # 5. Hacer el merge con el resumen
    resumen_completo = combinaciones.merge(
        resumen_v2,
        on=['dni', f'semestre_relativo_{cohorte}'],
        how='left'
    ).fillna(0)

    resumen_completo['total_actividad'] = (
        resumen_completo['inscripciones'] +
        resumen_completo['tp_aprobados'] +
        resumen_completo['finales_inscriptos']
    )

    resumen_completo['total_actividad_promedio'] = resumen_completo['total_actividad'] / 3

    metricas = ['inscripciones', 'tp_aprobados', 'finales_inscriptos']

    for metrica in metricas:
        tabla = pd.crosstab(resumen_completo[f'semestre_relativo_{cohorte}'],
                            resumen_completo[metrica])
        plt.figure(figsize=(10, 6))
        sns.heatmap(tabla, annot=True, fmt='d', cmap='Blues')
        plt.title(f'Distribución de {metrica} por semestre_relativo {cohorte}')
        plt.xlabel(metrica)
        plt.ylabel(f'semestre_relativo {cohorte}')
        plt.show()

    tabla = pd.crosstab(resumen_completo[f'semestre_relativo_{cohorte}'],
                         resumen_completo['total_actividad'])

    plt.figure(figsize=(10, 6))
    sns.heatmap(tabla, annot=True, fmt='d', cmap='Purples')
    plt.title('Distribución de actividad total por semestre_relativo')
    plt.xlabel('Total de Actividad (inscripciones + tp_aprobados + finales_inscriptos)')
    plt.ylabel(f'semestre_relativo {cohorte}')
    plt.show()
    
    tabla_2 = pd.crosstab(resumen_completo[f'semestre_relativo_{cohorte}'],
                         resumen_completo['total_actividad_promedio'])

    plt.figure(figsize=(10, 6))
    sns.heatmap(tabla_2, annot=True, fmt='d', cmap='Purples')
    plt.title('Distribución de actividad total promedio por semestre_relativo')
    plt.xlabel('Total de Actividad Promedio (inscripciones + tp_aprobados + finales_inscriptos)/3')
    plt.ylabel(f'semestre_relativo {cohorte}')
    plt.show()

    print(resumen_completo.groupby(f'semestre_relativo_{cohorte}').count())
    return resumen_completo

In [ ]:
resumen_completo_20221c = cuatro_metricas(actas_inscriptos_2022_cohorte_20221c, combinaciones_cohorte_20221c, '20221c')

Lo que quiero confiirmar es que todos los puntajes (1,2,3,4) bajan a medida que aumenta el semestre.
El unico que aumenta es puntaje 0.  Pero eso confirma el abandono

In [52]:
def promedio_cantidad_actividad_personas(resumen_completo, met, semestre_relativo, promedio_personas, actividad):
    # Filtrar los valores entre 1 y 3 actividades
    df_filtrado = resumen_completo[resumen_completo[met].isin(actividad)]

    # Contar cuántas personas por semestre y total_actividad
    conteo = df_filtrado.groupby([semestre_relativo, met])['dni'].nunique().reset_index(name='cantidad_personas')

    # Promediar por semestre
    promedio = conteo.groupby(semestre_relativo)['cantidad_personas'].mean().reset_index(name=promedio_personas)
    print(promedio)

In [ ]:
promedio_cantidad_actividad_personas(resumen_completo_20221c, 'total_actividad', 'semestre_relativo_20221c', 'promedio_personas_1_2', [1, 2])

In [ ]:
promedio_cantidad_actividad_personas(resumen_completo_20221c, 'total_actividad', 'semestre_relativo_20221c', 'promedio_personas_1_2_3', [1, 2, 3])

In [ ]:
promedio_cantidad_actividad_personas(resumen_completo_20221c, 'total_actividad', 'semestre_relativo_20221c', 'promedio_personas_1_2_3_4', [1, 2, 3, 4])

In [ ]:
promedio_cantidad_actividad_personas(resumen_completo_20221c, 'total_actividad', 'semestre_relativo_20221c', 'promedio_personas_1_2_3_4_5', [1, 2, 3, 4, 5])

In [ ]:
# 1. Clasificar la actividad
def clasificar_actividad(valor):
    if valor == 0:
        return 'sin_actividad'
    elif valor <= 3:
        return 'actividad_baja'
    else:
        return 'actividad_alta'

resumen_completo_20221c['categoria_actividad'] = resumen_completo_20221c['total_actividad'].apply(clasificar_actividad)

# 2. Contar por categoría y semestre
cuadro = resumen_completo_20221c.groupby(
    [f'semestre_relativo_20221c', 'categoria_actividad']
).agg(
    cantidad=('dni', 'nunique')  # suponiendo que cada fila es una persona por semestre
).reset_index()

# 3. Calcular total por semestre para sacar porcentaje
totales = cohorte_20221c.size
cuadro['porcentaje'] = cuadro['cantidad'] / totales * 100

# 4. Pivotear para que quede como tabla resumen
cuadro_pivot = cuadro.pivot(
    index=f'semestre_relativo_20221c',
    columns='categoria_actividad',
    values='porcentaje'
).fillna(0).reset_index()
cuadro_pivot

In [ ]:
resumen_completo_20221c

viendo todo esto para la cohorte 20222c

In [ ]:
resumen_completo_20222c = cuatro_metricas(actas_inscriptos_2022_cohorte_20222c, combinaciones_cohorte_20222c, '20222c')

In [ ]:
promedio_cantidad_actividad_personas(resumen_completo_20222c, 'total_actividad', 'semestre_relativo_20222c', 'promedio_personas_1_2', [1, 2])

In [ ]:
promedio_cantidad_actividad_personas(resumen_completo_20222c, 'total_actividad', 'semestre_relativo_20222c', 'promedio_personas_1_2_3', [1, 2, 3])

In [ ]:
promedio_cantidad_actividad_personas(resumen_completo_20222c, 'total_actividad', 'semestre_relativo_20222c', 'promedio_personas_1_2_3_4', [1, 2, 3, 4])

In [ ]:
# 1. Clasificar la actividad
def clasificar_actividad(valor):
    if valor == 0:
        return 'sin_actividad'
    elif valor <= 3:
        return 'actividad_baja'
    else:
        return 'actividad_alta'

resumen_completo_20222c['categoria_actividad'] = resumen_completo_20222c['total_actividad'].apply(clasificar_actividad)

# 2. Contar por categoría y semestre
cuadro = resumen_completo_20222c.groupby(
    [f'semestre_relativo_20222c', 'categoria_actividad']
).agg(
    cantidad=('dni', 'nunique')  # suponiendo que cada fila es una persona por semestre
).reset_index()

# 3. Calcular total por semestre para sacar porcentaje
totales = cohorte_20222c.size
cuadro['porcentaje'] = cuadro['cantidad'] / totales * 100

# 4. Pivotear para que quede como tabla resumen
cuadro_pivot = cuadro.pivot(
    index=f'semestre_relativo_20222c',
    columns='categoria_actividad',
    values='porcentaje'
).fillna(0).reset_index()
cuadro_pivot

In [ ]:
totales

# Veo un caso especial

In [ ]:
resumen_completo[resumen_completo['total_actividad'] == 17]

In [67]:
dni_caso_especial = resumen_completo[resumen_completo['total_actividad'] == 17]['dni'].values[0]

In [ ]:
# quiero ver toda la salida
# y no solo la primera fila
pd.set_option('display.max_columns', None)
personas_2022[personas_2022['dni'] == dni_caso_especial]

In [ ]:
actas_inscriptos_2022_cohorte_20221c[actas_inscriptos_2022_cohorte_20221c['dni'] == dni_caso_especial]['materia'].value_counts()

In [ ]:
actas_inscriptos_2022_cohorte_20221c[actas_inscriptos_2022_cohorte_20221c['dni'] == dni_caso_especial][actas_inscriptos_2022_cohorte_20221c['materia'] == 'Grandes Modelos de Lenguaje: Fundamentos y Aplicaciones - ECI2024'].value_counts()

In [ ]:
actas_inscriptos_2022_cohorte_20221c[actas_inscriptos_2022_cohorte_20221c['dni'] == dni_caso_especial][actas_inscriptos_2022_cohorte_20221c['semestre_relativo_20221c'] == 4]

Me gustaría ver también la cantidad de actas de examen que figuran como aprobadas

In [ ]:
# 1. Crear columna auxiliar para finales aprobados
actas_inscriptos_2022['es_final_aprobado'] = (
    (actas_inscriptos_2022['tipo_acta'] == 'Acta de Examen') &
    (actas_inscriptos_2022['resultado'] == 'Aprobado')
)

# 2. Agrupar y sumar
resumen_finales_aprobados = actas_inscriptos_2022.groupby(['dni', 'semestre_relativo']).agg(
    finales_aprobados=('es_final_aprobado', 'sum')
).reset_index()

# 4. Merge para completar con ceros
resumen_finales_aprobados_completo = combinaciones.merge(
    resumen_finales_aprobados,
    on=['dni', 'semestre_relativo'],
    how='left'
).fillna(0)

In [ ]:
tabla = pd.crosstab(resumen_finales_aprobados_completo['semestre_relativo'], resumen_finales_aprobados_completo['finales_aprobados'])
plt.figure(figsize=(10, 6))
sns.heatmap(tabla, annot=True, fmt='d', cmap='Blues')
plt.title(f'Distribución de finales aprobados por semestre_relativo')
plt.xlabel('finales_aprobados')
plt.ylabel('semestre_relativo')
plt.show()

Puedo calcular la procrastinación, dado que aprobó los TP, nos gustaría ver cuánto tarda en rendir el final en promedio

In [ ]:
# Paso 1: Separar las promociones aprobadas
promo_aprobadas = actas_inscriptos_2022[
    (actas_inscriptos_2022['tipo_acta'] == 'Acta de Regulares/Promociones') &
    (actas_inscriptos_2022['resultado'] == 'Aprobado')
][['dni', 'materia', 'semestre_relativo']].rename(
    columns={'semestre_relativo': 'semestre_promocion'}
)

# Paso 2: Separar los exámenes
actas_examen = actas_inscriptos_2022[
    (actas_inscriptos_2022['tipo_acta'] == 'Acta de Examen')
][['dni', 'materia', 'semestre_relativo']].rename(
    columns={'semestre_relativo': 'semestre_examen'}
)

# Paso 3: Hacer merge entre promociones aprobadas y exámenes
merged = promo_aprobadas.merge(actas_examen, on=['dni', 'materia'], how='inner')

# Paso 4: Calcular la diferencia de semestre
merged['diferencia_semestres'] = merged['semestre_examen'] - merged['semestre_promocion']

In [ ]:
conteo = merged['diferencia_semestres'].value_counts().sort_index()

# Crear el gráfico
plt.figure(figsize=(10, 6))
sns.barplot(x=conteo.index, y=conteo.values, color='skyblue')

plt.title('Distribución de Diferencia de Semestres entre Promoción y Examen')
plt.xlabel('Diferencia de Semestres')
plt.ylabel('Cantidad de Casos')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
promedio_por_dni = merged.groupby('dni')['diferencia_semestres'].mean().reset_index()
promedio_por_dni.columns = ['dni', 'promedio_diferencia_semestres']
plt.figure(figsize=(10, 6))
sns.histplot(promedio_por_dni['promedio_diferencia_semestres'], bins=20, kde=True, color='salmon')

plt.title('Distribución del Promedio de Diferencia de Semestres por DNI')
plt.xlabel('Promedio de Diferencia de Semestres')
plt.ylabel('Cantidad de Estudiantes')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

### Genero el EDA inicial

In [ ]:
helpers_eda_inicial.dataset_profiling(personas_2022, 'personas_2022')
helpers_eda_inicial.dataset_profiling(actas_inscriptos_2022, 'actas_inscriptos_2022')